# <strong>Tensor Flow Deep Learning
<strong>딥러닝과 텐서플로의 만남</strong>
1. <strong>규칙기반 알고리즘</strong> : for/ if/ boolean - 분명한 기준이 존재한 경우 효과적
2. <strong>머신러닝 알고리즘</strong> : end-to-end 학습

<br>
## <strong>1 Jupyter Notebook
쥬피터 노트북
### <strong> 01 tensor graph 설계
Tensor() : graph 구조를 결과로 출력한다

In [9]:
# constant : 상수, 문자형태도 입력가능
import tensorflow as tf
hello = tf.constant("Hello, Tensorflow!")
print(hello)

# 'rank'  : Tensor 차원 수
# 'shape' : Tensor 구조를 설명
# [1., 2., 3.]                    : 'rank' 1차원 tensor, 'shape' [3]     (인덱스 3)
# [[1., 2., 3.], [4., 5., 6.]]    : 'rank' 2차원 tensor, 'shape' [2,3]   (2차원 인덱스 3)
# [[[1., 2., 3.]],[[7., 8., 9.]]] : 'rank' 3차원 tensor, 'shape' [2,1,3] (3차원 2x1x3)

Tensor("Const_3:0", shape=(), dtype=string)


In [5]:
a = tf.constant(10)
b = tf.constant(32)
c = tf.add(a,b)
print(c)

Tensor("Add:0", shape=(), dtype=int32)


### <strong> 02 지연실행
lazy evaluation : graph 구조를 그린 뒤, Tensor 연산을 실행한다 

In [7]:
print(sess.run(hello))
print(sess.run([a,b,c]))
sess.close()

b'Hello, Tensorflow!'
[10, 32, 42]


<br>
## <strong>2 PlaceHolder 와 변수
parameter 를 설정
1. <strong>scalar</strong> : 크기 값  <strong>ex)1</strong>
2. <strong>vector</strong> : 화살표 1개 <strong>ex) [1,2,3] list 1개</strong>
3. <strong>matrix</strong> : 화살표들의 여러 묶음 <strong>ex) [[1,2],[3,4]]</strong>
4. <strong>shape</strong> : 다차원 자료를 x 형태로 표시

https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/resources/dims_types.html

<img src="https://cdn-images-1.medium.com/max/1600/1*7aXSK5bdSFO9WZMswU5v1w.png" align="left" width="700">

In [11]:
# Tensorflow 의 경직성을 입력 부분에 한해서 
# 조금 더 유연한 입력값을 설정가능

# None : rank 차원을 범용으로 설정
# 3 : feactor 는 3으로 고정하여 모델의 안정성 확보
X = tf.placeholder(tf.float32, [None, 3])  
print(X)

Tensor("Placeholder_1:0", shape=(?, 3), dtype=float32)


In [12]:
# tf.Variable: 그래프를 계산하면서 자료를 최적화
# tf.random_normal: 초기값을 정규분포 랜덤 값으로 초기화
W = tf.Variable(tf.random_normal([3, 2]))
b = tf.Variable(tf.random_normal([2, 1]))

# 가설함수 : 1차원 선형모델을 정의한다
expr = tf.matmul(X, W) + b
# ------------ graph 설계 완료 --------------------

<img src="http://cfile2.uf.tistory.com/image/226F5B3952B78BC7123696" align='left'>

In [19]:
# ------------ lazy evaluation -----------------
sess = tf.Session()
x_data = [[1, 2, 3], [4, 5, 6]]

# Variable 초기화 : tf.global_variables_initializer
sess.run(tf.global_variables_initializer())

# placeholder 에 입력한 값
print("x_data : {},\n\nW(weight) : {},\n\nb(bias) : {},\n\nexpr(예측) : {}".format(
    x_data, sess.run(W), sess.run(b), 
    
    # expr 실행
    # feed_dict : 'X'매개변수와 입력값 연결 {dict 을 활용}
    sess.run(expr, feed_dict={X: x_data})))
sess.close()

x_data : [[1, 2, 3], [4, 5, 6]],

W(weight) : [[ 0.62197512  0.20127779]
 [-0.06056291  1.12221813]
 [ 0.6104278   0.61923862]],

b(bias) : [[-0.2599577 ]
 [-1.26874745]],

expr(예측) : [[ 2.07217479  4.04347181]
 [ 4.57890511  8.86288643]]


<br>
## <strong>3 선형회귀 모델을 구현
선형회귀 : x 와 y 의 관계를 파악한다

In [20]:
# import tensorflow as tf
x_data = [1, 2, 3]
y_data = [1, 2, 3]

W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))

In [21]:
# name : 값의 추적을 위해 이름을 설정
# 변수, 연산함수에도 이름을 지정할 수 있다.
X = tf.placeholder(tf.float32, name="X")
Y = tf.placeholder(tf.float32, name="Y")
print(X)
print(Y)   # tensor 에도 이름이 붙여져서 출력

Tensor("X:0", dtype=float32)
Tensor("Y:0", dtype=float32)


In [22]:
# 가설 수식: y = W * x + b
# W 와 X 가 행렬이 아니므로, tf.matmul 아닌 기본 곱셈 기호를 사용
hypothesis = W * X + b

cost = tf.reduce_mean(tf.square(hypothesis - Y))  # 예측에서 실제를 뺀, 제곱값들의 평균 (평균제곱오차)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1) # 경사하강법
train_op = optimizer.minimize(cost)                              # 비용 최소화 목표

In [29]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())  # 세션을 생성하고 초기화합니다.

    for step in range(100): # 최적화를 100번 수행
        # sess.run : 1) train_op 와 cost 그래프를 계산, 2) 가설수식 값을 feed_dict 로 전달
        _, cost_val = sess.run([train_op, cost], feed_dict={X: x_data, Y: y_data})
        if step % 20 == 0: 
            print(step, cost_val, sess.run(W), sess.run(b))

    # 최적화가 완료된 모델에 테스트 값을 넣고 결과가 잘 나오는지 확인해봅니다.
    print("\n=== Test ===\nX: 5, Y:{} \nX: 2.5, Y:{}".format(
        sess.run(hypothesis, feed_dict={X: 5}),     # X 가 5일때 Y를 예측
        sess.run(hypothesis, feed_dict={X: 2.5})))  # X 가 2.5일때 Y를 예측

0 15.8906 [ 0.883205] [ 0.72400987]
20 0.0240177 [ 0.82433146] [ 0.39933595]
40 0.00907453 [ 0.8920207] [ 0.24546234]
60 0.0034286 [ 0.93362767] [ 0.15087987]
80 0.00129542 [ 0.95920247] [ 0.09274228]

=== Test ===
X: 5, Y:[ 4.92993593] 
X: 2.5, Y:[ 2.49417329]


<br>
## <strong>4 기본 신경망 구현
뉴런 신경망을 연결한 딥러닝 알고리즘 
1. ex) $ y = Sigmoid(X * W + b) $
1. <strong>인공뉴런</strong> : <strong>weight</strong>(가중치) 와 <strong>활성화 함수</strong>의 연결로 구성
1. <strong>activation function</strong> (활성화 함수) : Sigmoid, ReLU, tanh

<img src="https://ujwlkarn.files.wordpress.com/2016/08/screen-shot-2016-08-08-at-11-53-41-am.png?w=748" align="left" 
\width="700">

<img src="http://python3.codes/wp-content/uploads/2017/01/XIHOY.jpg" align="left" width="700">

In [30]:
# RBM (Restricted Boltzmann Machine) : 제한된 볼트만 머신
# 역전파 (backpropagation) : 출력층 결과를 입력층까지 역으로 전파하며 계산하여 최적화 과정이 훨씬 유용하다
# 털과 날개 여부로, 포유류/조류 분류 신경망모델 생성

import tensorflow as tf
import numpy as np
x_data = np.array([[0, 0], [1, 0], [1, 1], 
                   [0, 0], [0, 0], [0, 1]])   # 0: 털, 1: 날개 

# one-hot-encoding : 기타([1, 0, 0]), 포유류([0, 1, 0]), 조류([0, 0, 1])
y_data = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], 
                   [1, 0, 0], [1, 0, 0], [0, 0, 1]])

In [ ]:
# 1. 신경망 모델 구성 graph ------------------------------------------------
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 신경망 (2차원) : [Train 데이터 특성, Target 레이블] -> [2, 3]
# 자료 x, y 모두 6차원으로 동일해야 한다
W = tf.Variable(tf.random_uniform([2, 3], -1., 1.)) # tf.random_uniform : 균등분포 난수출력 -1 ~ 1
b = tf.Variable(tf.zeros([3]))  # bias : 각 레이어의 아웃풋 갯수로 설정

In [ ]:
L = tf.add(tf.matmul(X, W), b)  # 신경망에 가중치 W과 편향 b을 적용합니다
L = tf.nn.relu(L)               # ReLU 함수를 적용
model = tf.nn.softmax(L)        # softmax 함수 : 결과 전체 합이 1인 확률로 만들어주는 함수
                                # 예) [8.04, 2.76, -6.52] -> [0.53 0.24 0.23]

# 신경망을 최적화하기 위한 비용 함수 (Cross-Entropy)
# cf) axis 옵션 : 개별 결과를 구한 뒤 평균 계산한다 (axis 이 없으면 -1.09 같은 총합 스칼라로 출력)
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))
#        Y         model           Y * tf.log(model)    reduce_sum(axis=1)
# 예) [[1 0 0]  [[0.1 0.7 0.2]  -> [[-1.0  0    0]   -> [-1.0, -0.09]
#     [0 1 0]]  [0.2 0.8 0.0]]     [ 0   -0.09 0]]

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

In [31]:
# 신경망을 최적화하기 위한 비용 함수를 작성 (Cross-Entropy)
# cf) axis 옵션 : 개별 결과를 구한 뒤 평균 계산한다 (axis 이 없으면 -1.09 같은 총합 스칼라로 출력)
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

#        Y         model           Y * tf.log(model)   reduce_sum(axis=1)
# 예) [[1 0 0]  [[0.1 0.7 0.2]  -> [[-1.0  0    0]  -> [-1.0, -0.09]
#     [0 1 0]]  [0.2 0.8 0.0]]     [ 0   -0.09 0]]

In [32]:
# 2. 신경망 모델 Train -------------------------------------------------------
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    if (step + 1) % 20 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

# 결과 확인  0: 기타 1: 포유류, 2: 조류
# tf.argmax: 예측값과 실제값의 행렬에서 tf.argmax 를 이용해 가장 큰 값을 가져옵니다.
# 예) [[0 1 0] [1 0 0]] -> [1 0]
#    [[0.2 0.7 0.1] [0.9 0.1 0.]] -> [1 0]

20 1.16454
40 1.15386
60 1.1436
80 1.13373
100 1.12424


In [33]:
# 3. 신경망 모델 의 검증 -------------------------------------------------------
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 1 1 0 0 1]
실제값: [0 1 2 0 0 2]
정확도: 66.67
